READ IMAGES FROM "SAMPLES" folder, in sequence.

Create a NumPy archieve (NPZ is a file format by numpy that provides storage of array data using gzip compression) named "sequence_array.npz", containing the sequence of the images from them 'Samples' folder, as a Numpy array.

(Requires packages: Pillow, Tensorflow=1.15, Keras)
(conda install Pillow tensorflow=1.15 keras)

----THIS IS BASED ON TF 2.0 ----

In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
import time

IMAGE_PATH = './samples/'

WIDTH = 100
HEIGHT = 100
SEQUENCE = np.array([])
BASIC_SEQUENCE = np.array([])
NEXT_SEQUENCE = np.array([])
NUMBER = 0

def image_initialize(image):
    picture = Image.open(image)
    picture = picture.crop((243, 176, 1428, 1280))
    picture = picture.resize((WIDTH, HEIGHT), Image.ANTIALIAS)
    picture = picture.convert('L')
    picture.save('./temp/1.png')
    data = np.array(picture.getdata()).reshape(WIDTH, HEIGHT, 1)
    return data

for file in os.listdir(IMAGE_PATH):
    image_array = image_initialize(os.path.join(IMAGE_PATH, file))
    SEQUENCE = np.append(SEQUENCE, image_array)
    NUMBER += 1
    print(NUMBER)
    print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

SEQUENCE = SEQUENCE.reshape(NUMBER, WIDTH * HEIGHT)
for i in SEQUENCE:
    for j in range(int(len(i))):
        if i[j] < 50:
            i[j] = 0


np.savez('sequence_array.npz', sequence_array=SEQUENCE)
print('Data saved.')
print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))

1
2020-02-06 13:43:38
2
2020-02-06 13:43:38
3
2020-02-06 13:43:38
4
2020-02-06 13:43:38
5
2020-02-06 13:43:38
6
2020-02-06 13:43:39
7
2020-02-06 13:43:39
8
2020-02-06 13:43:39
9
2020-02-06 13:43:39
10
2020-02-06 13:43:39
11
2020-02-06 13:43:39
12
2020-02-06 13:43:39
13
2020-02-06 13:43:39
14
2020-02-06 13:43:39
15
2020-02-06 13:43:39
16
2020-02-06 13:43:39
17
2020-02-06 13:43:39
18
2020-02-06 13:43:39
19
2020-02-06 13:43:39
20
2020-02-06 13:43:39
21
2020-02-06 13:43:40
22
2020-02-06 13:43:40
23
2020-02-06 13:43:40
24
2020-02-06 13:43:40
25
2020-02-06 13:43:40
26
2020-02-06 13:43:40
27
2020-02-06 13:43:40
28
2020-02-06 13:43:40
29
2020-02-06 13:43:40
30
2020-02-06 13:43:40
31
2020-02-06 13:43:40
32
2020-02-06 13:43:40
33
2020-02-06 13:43:40
34
2020-02-06 13:43:40
35
2020-02-06 13:43:40
36
2020-02-06 13:43:41
37
2020-02-06 13:43:41
38
2020-02-06 13:43:41
39
2020-02-06 13:43:41
40
2020-02-06 13:43:41
41
2020-02-06 13:43:41
42
2020-02-06 13:43:41
43
2020-02-06 13:43:41
44
2020-02-06 13:43:

In [5]:
# Check the array with the images
# opening the archive and accessing each array by name
with np.load("sequence_array.npz") as my_archive_file:
    out0 = my_archive_file["sequence_array"]
print(out0)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## MODEL TO TRAIN THE CONVLSTM ##


In [9]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import os
import time




'''
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import matplotlib.pyplot as plt
'''

'''
from keras.utils import multi_gpu_model
from keras import optimizers
'''


'\nfrom keras.utils import multi_gpu_model\nfrom keras import optimizers\n'

In [ ]:
WIDTH = 100
HEIGHT = 100
FRAMES = 16

SEQUENCE = np.load('sequence_array.npz')['sequence_array']  # load array
print(SEQUENCE[16])
print('Data loaded.')

#print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
NUMBER = len(SEQUENCE)
# step =1
SEQUENCE = SEQUENCE.reshape(NUMBER, WIDTH, HEIGHT, 1)

# step =2
SEQUENCE_2 = []
for i in range(int(NUMBER / 2)):
    SEQUENCE_2.append(SEQUENCE[2 * i])

# step = 3
SEQUENCE_3 = []
for i in range(int(NUMBER / 3)):
    SEQUENCE_3.append(SEQUENCE[3 * i])

In [ ]:
'''
SEQUENCE = SEQUENCE.reshape(NUMBER, WIDTH, HEIGHT, 1)
print(SEQUENCE)
BASIC_SEQUENCE = np.zeros((NUMBER-FRAMES, FRAMES, WIDTH, HEIGHT, 1))
print(BASIC_SEQUENCE)
NEXT_SEQUENCE = np.zeros((NUMBER-FRAMES, FRAMES, WIDTH, HEIGHT, 1))
print(NEXT_SEQUENCE)
'''

In [10]:
for i in range(FRAMES):
    #plt.imshow(BASIC_SEQUENCE[0][0].reshape(100, 100))
    #plt.show()
    
    model = Sequential()

    model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),input_shape=(None, WIDTH, HEIGHT, 1), padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(ConvLSTM2D(filters=60, kernel_size=(3, 3), padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(ConvLSTM2D(filters=60, kernel_size=(3, 3), padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3), padding='same', return_sequences=True))
    model.add(BatchNormalization())

    model.add(Conv3D(filters=1, kernel_size=(3, 3, 3), activation='sigmoid', padding='same', data_format='channels_last'))
    
     
    # For a multi-class classification problem (https://keras.io/getting-started/sequential-model-guide/)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

    # Model Summary; Training parameters etc...
    model.summary()


NameError: name 'FRAMES' is not defined

#### TEST MODEL ####

In [14]:
seq = models.Sequential()

seq.add(layers.ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   input_shape=(None, 40, 40, 1),
                   padding='same', return_sequences=True))
seq.add(layers.BatchNormalization())

seq.add(layers.ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(layers.BatchNormalization())

seq.add(layers.ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(layers.BatchNormalization())

seq.add(layers.ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(layers.BatchNormalization())

seq.add(layers.Conv3D(filters=1, kernel_size=(3, 3, 3),
               activation='sigmoid',
               padding='same', data_format='channels_last'))
seq.compile(loss='binary_crossentropy', optimizer='adadelta')

######### TEST ###########
 FAKE IMAGE GENERATION
#########################

In [15]:
def generate_movies(n_samples=1200, n_frames=15):
    row = 80
    col = 80
    noisy_movies = np.zeros((n_samples, n_frames, row, col, 1), dtype=np.float)
    shifted_movies = np.zeros((n_samples, n_frames, row, col, 1),
                              dtype=np.float)

    for i in range(n_samples):
        # Add 3 to 7 moving squares
        n = np.random.randint(3, 8)

        for j in range(n):
            # Initial position
            xstart = np.random.randint(20, 60)
            ystart = np.random.randint(20, 60)
            # Direction of motion
            directionx = np.random.randint(0, 3) - 1
            directiony = np.random.randint(0, 3) - 1

            # Size of the square
            w = np.random.randint(2, 4)

            for t in range(n_frames):
                x_shift = xstart + directionx * t
                y_shift = ystart + directiony * t
                noisy_movies[i, t, x_shift - w: x_shift + w,
                             y_shift - w: y_shift + w, 0] += 1

                # Make it more robust by adding noise.
                # The idea is that if during inference,
                # the value of the pixel is not exactly one,
                # we need to train the network to be robust and still
                # consider it as a pixel belonging to a square.
                if np.random.randint(0, 2):
                    noise_f = (-1)**np.random.randint(0, 2)
                    noisy_movies[i, t,
                                 x_shift - w - 1: x_shift + w + 1,
                                 y_shift - w - 1: y_shift + w + 1,
                                 0] += noise_f * 0.1

                # Shift the ground truth by 1
                x_shift = xstart + directionx * (t + 1)
                y_shift = ystart + directiony * (t + 1)
                shifted_movies[i, t, x_shift - w: x_shift + w,
                               y_shift - w: y_shift + w, 0] += 1

    # Cut to a 40x40 window
    noisy_movies = noisy_movies[::, ::, 20:60, 20:60, ::]
    shifted_movies = shifted_movies[::, ::, 20:60, 20:60, ::]
    noisy_movies[noisy_movies >= 1] = 1
    shifted_movies[shifted_movies >= 1] = 1
    return noisy_movies, shifted_movies

In [16]:
noisy_movies, shifted_movies = generate_movies(n_samples=1200)
noisy_movies.shape, shifted_movies.shape

((1200, 15, 40, 40, 1), (1200, 15, 40, 40, 1))

In [18]:
#### TRAIN THE NETWORK #####
%time
seq.fit(noisy_movies[:1000], shifted_movies[:1000], batch_size=10, epochs=5, validation_split=0.05)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs
Train on 950 samples, validate on 50 samples
Epoch 1/5
950/950 [==============================] - 872s 918ms/sample - loss: 0.9204 - val_loss: 0.7229
Epoch 2/5
950/950 [==============================] - 867s 913ms/sample - loss: 0.7839 - val_loss: 0.6978
Epoch 3/5
950/950 [==============================] - 902s 950ms/sample - loss: 0.7439 - val_loss: 0.6191
Epoch 4/5
950/950 [==============================] - 914s 962ms/sample - loss: 0.7137 - val_loss: 0.5619
Epoch 5/5
950/950 [==============================] - 890s 937ms/sample - loss: 0.6880 - val_loss: 0.5599


In [2]:
'''
Testing the network on one movie
Feed it with the first 7 positions and then predict the new positions
'''

which = 1004
track = noisy_movies[which][:7, ::, ::, ::]
track.shape, track[np.newaxis, ::, ::, ::, ::].shape
for j in range(16):
    new_pos = seq.predict(track[np.newaxis, ::, ::, ::, ::]) # (1, 7, 40, 40, 1)
    new = new_pos[::, -1, ::, ::, ::] # (1, 40, 40, 1)
    track = np.concatenate((track, new), axis=0) # adds +1 to the first dimension in each loop cycle

NameError: name 'noisy_movies' is not defined

In [1]:
'''
####  COMPARE PREDICTIONS WITH GROUND TRUTH ###
'''

track2 = noisy_movies[which][::, ::, ::, ::]

for i in range(15):
    fig = plt.figure(figsize=(10, 5))

    ax = fig.add_subplot(121)

    if i >= 7:
        ax.text(1, 3, 'Predictions !', fontsize=20, color='w')
    else:
        ax.text(1, 3, 'Initial trajectory', fontsize=20)

    toplot = track[i, ::, ::, 0]

    plt.imshow(toplot)
    ax = fig.add_subplot(122)
    plt.text(1, 3, 'Ground truth', fontsize=20)

    toplot = track2[i, ::, ::, 0]
    if i >= 2:
        toplot = shifted_movies[which][i - 1, ::, ::, 0]

    plt.imshow(toplot)
    plt.savefig('%i_animate.png' % (i + 1))

NameError: name 'noisy_movies' is not defined